# Running a network

The purpose of this notebook is to showcase how to train, save, and re-load a network, as well as how to produce predictions once training is done.

For how to build a plant from scratch, feel free to look up the `1-build-plant.ipynb` notebook.
For how to build a network, feel free to look up the `2-build-net.ipynb` notebook.

Let's start by importing what we need.


In [1]:

import os
import sys
import json
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

file_dir = sys.path[0]
sys.path.append(os.path.dirname(file_dir[:file_dir.rfind('tutorials')]))

import motornet as mn

%load_ext autoreload
%autoreload 2

print('\n')
print('tensorflow version: ' + tf.__version__)
print('numpy version: ' + np.__version__)




tensorflow version: 2.7.0
numpy version: 1.21.5


<Figure size 432x288 with 0 Axes>


# I. Introduction

Since the purpose of this notebook is not to show how to build a plant, we will use a pre-built plant that comes with the `motornet` toolbox. This is a 4-muscles point mass plant, with `ReluMuscle` actuators.

We will also use the default `motornet` network, which is a GRU network. Since we only specify an integer (rather than a list of integers) for `n_units`, this will end up being a one-layer GRU network.

We will also recreate a network similar to that created in `2-build-net.ipynb`


In [77]:

def create_network():
  plant = mn.plants.ReluPointMass24(proprioceptive_delay=0.3)
  network = mn.nets.layers.GRUNetwork(plant=plant, n_units=50, kernel_regularizer=10**-6, name='network')
  task = mn.tasks.CentreOutReach(network=network)
  rnn = tf.keras.layers.RNN(cell=network, return_sequences=True, name='RNN')

  input_dict = task.get_input_dict_layers()
  state0 = task.get_initial_state_layers()
  states_out = rnn(input_dict, initial_state=state0)

  nn = mn.nets.MotorNetModel(inputs=[input_dict, state0], outputs=states_out, name='model', task=task)
  nn.compile(optimizer=tf.optimizers.Adam(clipnorm=1.), loss=task.losses, loss_weights=task.loss_weights)
  return nn

model = create_network()
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, None, 5)]    0           []                               
                                                                                                  
 joint0 (InputLayer)            [(None, 4)]          0           []                               
                                                                                                  
 cartesian0 (InputLayer)        [(None, 4)]          0           []                               
                                                                                                  
 muscle0 (InputLayer)           [(None, 4, 4)]       0           []                               
                                                                                              

# II. Training the network

The process for training a network is sensibly similar to that of `tensorflow`, from which `motornet` is built.


In [ ]:

n_t = 100
n_batches = 3000
batch_size = 32

# this callback logs training information for each batch passed, rather than for each epoch.
callbacks = [mn.nets.callbacks.BatchLogger()]

[inputs, targets, init_states] = model.task.generate(n_timesteps=n_t, batch_size=n_batches * batch_size)
h = model.fit(x=[inputs, init_states], y=targets, verbose=1, epochs=1, batch_size=batch_size, shuffle=False, callbacks=callbacks)




2483/3000 [=======================>......] - ETA: 1:12 - loss: 0.1436 - position_loss: 0.1253 - gru_regularizer_loss: 0.1606 - l2_xdx_activation_loss: 4.3536e-04


Using the training log, stored in the `BatchLogger` callback object, we can plot all losses over training iterations.


In [ ]:

def print_training_log(log):
  plt.figure().set_tight_layout(True)
  for kk, vv in log.items():
    plt.plot(vv, label=kk)

  plt.ylabel("Loss")
  plt.xlabel("Batch #")
  plt.legend()
  plt.show()

training_log = callbacks[0].history
print_training_log(log=training_log)


# III. Evaluating performance

Again, this is fairly similar to what one would expect from a `tensorflow` model.
A forward pass on the model (with `training=False`) will yield a dictionary containing all the states over time.


In [ ]:
[inputs, targets, init_states] = model.task.generate(n_timesteps=n_t, batch_size=32)
results = model([inputs, init_states], training=False)

print("returned shape of each state:")
print("-----------------------------")
for k, v in results.items():
  print(k + ": ", v.shape)


We can plot these states to quickly visualize what the network is doing.


In [ ]:

plotor = mn.utils.plotor.plot_pos_over_time

def plot_simulations(xy, target_xy):
  target_x = target_xy[:, -1, 0]
  target_y = target_xy[:, -1, 1]

  plt.figure(figsize=(15,5))

  plt.subplot(1,2,1)
  plotor(xy)
  plt.scatter(target_x, target_y)

  plt.subplot(1,2,2)
  plotor(xy - target_xy)
  plt.axhline(0, c="grey")
  plt.axvline(0, c="grey")
  plt.xlabel("X distance to target")
  plt.ylabel("Y distance to target")
  plt.show()

plot_simulations(xy=results["cartesian position"], target_xy=targets)


# IV. Saving the model

Next, we can save the model using built-in methods. We will also save the training history as a `.json` file.


In [ ]:

weight_file = "save" + os.path.sep + "weights"
log_file = "save" + os.path.sep + "log.json"
data_file = "save" + os.path.sep + "data.pickle"
cfg_file = "save" + os.path.sep + "cfg"

# save model weights
model.save_weights(weight_file)

# save model configuration
model.save_model(cfg_file)

# save training history (log)
with open(log_file, 'w') as file:
    json.dump(training_log, file)

print("Done.")


# V. Loading the model

## V. 1. The configuration file
This file is what the `model.save_model` method produces. It contains a dictionary with all the parameter and configuration values used to create the model. Let's look into its structure quickly.


In [ ]:


with open(cfg_file + ".json", 'r') as file:
    cfg = json.load(file)


for k1, v1 in cfg.items():
  print(k1 + ":")
  for k2, v2 in v1.items():
    if type(v2) is dict:
      print("\t\t" + k2 + ":")
      for k3, v3 in v2.items():
        print("\t\t\t\t" + k3 + ": ", v3)
    else:
      print("\t\t" + k2 + ": ", v2)



## V. 2. Training history

We can use the `.json` file containing the training history to retrieve and plot losses over time.

In [ ]:

# retrieve and plot training history
with open(log_file, 'r') as file:
    loaded_training_log = json.load(file)

print_training_log(log=loaded_training_log)




## V. 3. The model itself

Unfortunately there is no straightforward, one-liner way to save a model. This is to an extent due to limitations from the `tensorflow` saving routines, and also due to the fact that we use fairly non-conventional models, with plants, tasks, etc... nested into one object. Usually, reloading the model would involve re-creating the same model, and applying the trained weights to that model again. To re-create the model, one could re-use the same function as was used to create the original model (that's what we will do here), or one could exploit the configuration file we saved and re-loaded above to recreate a model exhaustively. The latter method might be more tedious to implement but has the benefit of being more exhaustive on the long run.

Note that the task object is contained in the network and so can be retried from there.


In [ ]:

# The model has to be re-created
model2 = create_network()
task2 = model2.task

# We then load the weights obtained from the training process
model2.load_weights(weight_file).expect_partial()




# VI. Evaluate the loaded model

We can then do a forward pass the loaded model like we did on the original model.

We can also save the data resulting from the forward pass, for later analyses if desired. This could have been done following the evaluation of the original model as well, obviously.


In [ ]:
[inputs, targets, init_states] = task2.generate(n_timesteps=n_t, batch_size=32)

results = model2([inputs, init_states], training=False)

# convert results to numpy array for saving later.
# (this is not mandatory)
inputs_np = {key: np.array(val) for key, val in inputs.items()}
results_np = {key: np.array(val) for key, val in results.items()}

# save as a .pickle file
with open(data_file, 'wb') as file:
  pickle.dump(
    obj={'results': results_np,
         'inputs': inputs_np,
         'targets': np.array(targets),
         'weights': model.get_weights(),
         'training_log': training_log,
         'task': cfg['Task'],
         'controller': cfg['Network'],
         'plant': cfg['Plant']},
    file=file)
  file.close()

print("Done.")


# VII. Analyze saved data

We can reload the saved data to perform analyses on it at any later time.

In [ ]:


# open data file
with open(data_file, 'rb') as file:
  data = pickle.load(file)

# plot data from data file
plot_simulations(xy=data["results"]["cartesian position"], target_xy=data["targets"])
